## POS tagging using modified Viterbi

### Data Preparation

In [1]:
#Importing libraries
import nltk
import numpy as np
import pandas as pd
import time, re, random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

In [2]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [3]:
# first few tagged sentences
print(nltk_data[:10])

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], [('Rudolph', 'NOUN'), ('Agnew', 'NOUN'), (',', '.'), ('55', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), ('and', 'CONJ'), ('former', 'ADJ'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Consolidated', 'NOUN'), ('Gold', 'NOUN'), ('Fields', 'NOUN'), ('PLC', 'NOUN'), (',', '.'), ('was', 'VERB'), ('named', 'VERB'), ('*-1', 'X'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('of', 'ADP'), ('this', 'DET'), ('British', 'ADJ'), ('industrial', 'ADJ'), ('

In [4]:
# Splitting into train and validation sets 95:5
random.seed(1234)
train_set, val_set = train_test_split(nltk_data, test_size=0.05, random_state=100)

print(len(train_set))
print(len(val_set))
print(train_set[:10])

3718
196
[[('One', 'NUM'), ('bright', 'ADJ'), ('sign', 'NOUN'), ('is', 'VERB'), ('that', 'ADP'), ('a', 'DET'), ('growing', 'VERB'), ('number', 'NOUN'), ('of', 'ADP'), ('women', 'NOUN'), ('have', 'VERB'), ('entered', 'VERB'), ('the', 'DET'), ('once', 'ADV'), ('male-dominated', 'ADJ'), ('field', 'NOUN'), (';', '.'), ('more', 'ADJ'), ('than', 'ADP'), ('a', 'DET'), ('third', 'ADJ'), ('of', 'ADP'), ('the', 'DET'), ('ringers', 'NOUN'), ('today', 'NOUN'), ('are', 'VERB'), ('women', 'NOUN'), ('.', '.')], [('``', '.'), ('These', 'DET'), ('days', 'NOUN'), (',', '.'), ('banking', 'NOUN'), ('customers', 'NOUN'), ('walk', 'VERB'), ('in', 'ADP'), ('the', 'DET'), ('door', 'NOUN'), ('*-1', 'X'), ('expecting', 'VERB'), ('you', 'PRON'), ('to', 'PRT'), ('have', 'VERB'), ('a', 'DET'), ('package', 'NOUN'), ('especially', 'ADV'), ('for', 'ADP'), ('them', 'PRON'), (',', '.'), ("''", '.'), ('Ms.', 'NOUN'), ('Moore', 'NOUN'), ('says', 'VERB'), ('*T*-2', 'X'), ('.', '.')], [('The', 'DET'), ('rights', 'NOUN'), (

In [5]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

95949

In [6]:
# Get the vocabulary 
tokens = [pair[0] for pair in train_tagged_words]
V = set(tokens)
print(len(V))

12106


In [7]:
# number of tags
T = set([pair[1] for pair in train_tagged_words])
len(T)

12

### POS tagging algorithm - HMM

Given a sequence of words, HMM algorithm assigns the most probable tag to the word. In other words, to every word w, assign the tag t that maximises the likelihood P(t/w). Since P(t/w) = P(w/t). P(t) / P(w), after ignoring P(w), we have to compute P(w/t) and P(t).

P(w/t) is basically the probability that given a tag (say NN), what is the probability of it being w (say 'building'). This can be computed by computing the fraction of all NNs which are equal to w, i.e. 

P(w/t) = count(w, t) / count(t). 

The term P(t) is the probability of tag t

#### Compute Emission Probabilities

In [8]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

# program being NOUN
print("\n", "program")
print(word_given_tag('program', 'NOUN'))


 program
(120, 27539)


#### Compute Transition Probabilities

In [9]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability

def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [10]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [11]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))

tags_df

,NUM,CONJ,DET,PRT,VERB,NOUN,PRON,.,ADP,ADV,ADJ,X
NUM,0.184195,0.013072,0.003862,0.026144,0.016934,0.352347,0.001485,0.118835,0.035056,0.002674,0.033571,0.211824
CONJ,0.042188,0.000464,0.118683,0.003709,0.155308,0.350487,0.058414,0.035698,0.053778,0.053778,0.118683,0.008809
DET,0.021640,0.000481,0.005771,0.000240,0.040394,0.637293,0.003727,0.017913,0.009618,0.012623,0.204977,0.045323
PRT,0.056102,0.002297,0.101050,0.001969,0.405184,0.245735,0.017717,0.043635,0.019357,0.010171,0.083661,0.013123
VERB,0.022448,0.005186,0.133292,0.031427,0.168744,0.111386,0.035916,0.034291,0.091184,0.082050,0.065640,0.218438
NOUN,0.009550,0.042921,0.013363,0.043357,0.146955,0.264280,0.004721,0.239951,0.177058,0.016813,0.012165,0.028868
PRON,0.007280,0.004981,0.009195,0.012261,0.484291,0.211494,0.007663,0.040613,0.023372,0.034100,0.072031,0.092720
.,0.081353,0.058032,0.173558,0.002511,0.088708,0.222531,0.065208,0.092923,0.092206,0.052292,0.043681,0.026908
ADP,0.061910,0.000848,0.323969,0.001484,0.008481,0.321213,0.069119,0.039754,0.017492,0.013357,0.107389,0.034984
ADV,0.031624,0.006991,0.069907,0.014314,0.344541,0.031624,0.015646,0.135153,0.119507,0.077230,0.130160,0.023302


### Build the vanilla Viterbi based POS tagger

In [12]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

#### Evaluation on validation set

In [13]:
# Running on entire validation dataset would take more than 3-4hrs. 
# Let's validation our Viterbi algorithm on a few sample sentences of validation  dataset

random.seed(1234)
# choose random 25 sents
#rndom = [random.randint(1,len(val_set)) for x in range(25)]

# list of sents
#val_run = [val_set[i] for i in rndom]
val_run = [sent for sent in val_set]

# list of tagged words
val_run_base = [tup for sent in val_run for tup in sent]

# list of untagged words
val_tagged_words = [tup[0] for sent in val_run for tup in sent]

In [14]:
# tagging the validation sentences
start = time.time()
tagged_seq = Viterbi(val_tagged_words)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)

# accuracy
check = [i for i, j in zip(tagged_seq, val_run_base) if i == j]  

accuracy = len(check)/len(tagged_seq)
print("Accuracy of the tagger: ", accuracy)

Time taken in seconds:  1590.2958076000214
Accuracy of the tagger:  0.91030251745293


In [15]:
# Incorrectly tagged words using vanilla Viterbi based POS tagger
incorrect_tagged_cases = [[val_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, val_run_base)) if j[0]!=j[1]]
incorrect_tagged_cases

[[('to', 'PRT'), (('book', 'NOUN'), ('book', 'VERB'))],
 [('leaving', 'VERB'), (('stocks', 'NOUN'), ('stocks', 'ADV'))],
 [('stocks', 'ADV'), (('up', 'PRT'), ('up', 'ADP'))],
 [('carried', 'VERB'), (('over', 'ADP'), ('over', 'PRT'))],
 [('apparently', 'ADV'), (('ignored', 'NUM'), ('ignored', 'VERB'))],
 [('Chilean', 'ADJ'), (('mine', 'NOUN'), ('mine', 'ADJ'))],
 [('the', 'DET'), (('Palestinian', 'ADJ'), ('Palestinian', 'NOUN'))],
 [('committee', 'NOUN'), (('first', 'ADJ'), ('first', 'ADV'))],
 [('.', '.'), (('Preston', 'NUM'), ('Preston', 'NOUN'))],
 [('Foster', 'NOUN'), (('Birmingham', 'NUM'), ('Birmingham', 'NOUN'))],
 [(',', '.'), (('Ala', 'NUM'), ('Ala', 'NOUN'))],
 [('has', 'VERB'), (('clamped', 'NUM'), ('clamped', 'VERB'))],
 [('their', 'PRON'), (('ankle', 'NUM'), ('ankle', 'NOUN'))],
 [('the', 'DET'), (('third-largest', 'NUM'), ('third-largest', 'ADJ'))],
 [('the', 'DET'), (('fifth-largest', 'NUM'), ('fifth-largest', 'ADJ'))],
 [('Charles', 'NOUN'), (('Z.', 'NUM'), ('Z.', 'NOUN'

#### Find unknown words in validation set

In [16]:
# vocabulary in val set
voc_val_set = set(val_tagged_words)

# Unknown words in val set
unknown_words = (voc_val_set-V)
print(unknown_words)

{'movies', 'initiate', 'preclinical', 'disapproval', 'Moon', '*T*-117', 'intelligent', '550,000', '618.1', 'display', 'manpower', '62-year-old', 'high-rise', 'dreamed', '11.95', 'diagram', 'Trans', 'faultlessly', 'R.D.', 'loose', 'applicable', 'Aerospace', 'CoreStates', 'Somerset', 'NCNB', 'adults', 'clouding', 'round-trip', 'shipboard', 'cardiovascular', '306', '88.32', 'Old-House', 'le', 'ECONOMIC', '*-165', 'glory', 'descending', 'Aquino', '*-152', 'Five', 'nonresidential', 'highest-pitched', '13.90', 'Chilver', 'Ilminster', 'intervention', 'escape', 'facade', 'Z.', 'shop', 'intricate', 'directorship', 'accordance', 'Video', '609', 'heebie-jeebies', '63-year-old', 'wrath', 'diagnosed', 'cleaner-burning', 'wealthy', 'reclaimed', 'Mont', 'refer', 'Birmingham', 'leveraging', 'DeFazio', 'viewpoints', 'flirted', 'fleet', 'Corazon', 'masters', 'insanity', 'clamped', 'hotels', 'S.A', 'relevance', 'amazingly', 'owning', 'blinks', 'Four', 'broadened', '858,000', '*T*-174', 'Researchers', 'aw

#### Unknown word observations
1. The unknown words like 'outlawing', 'safeguarding' and 'Indexing' which are verbs most of the times are tagged as 'NOUN'
2. The unknown words like 'broadened', 'shopped' and 'delayed' which are verbs most of the times are tagged as 'NOUN'
3. The unknown words like 'argues' and 'illustrates' which are verbs most of the times are tagged as 'NOUN'
4. The unknown words like 'third-largest' and 'finest' which are adjectives most of the times are tagged as 'NOUN'
5. The unknown words like 'broadly' and 'faultlessly' which are adverbs most of the times are tagged as 'NOUN'
6. Most of the decimal numbers are tagged as 'NOUN' instead of 'NUM'

#### Handling unknown words  

The Viterbi algorithm tags the unknown words with the most frequently occuring POS tag('PRON') as the emission probability is zero.

The Performance of the POS tagger can be improved by modifying the viterbi algorithm in two ways listed below.
1. By using rule based tagger for unknown words using morphological rules
2. By using the probability based technique, forcing emission probability to a small value (i.e 0.001) 

### Solve the problem of unknown words

In [17]:
# Modified Viterbi Heuristic with rule based backoff
def Viterbi_backedoff_with_rule_based(words, train_bag = train_tagged_words, vocabulary=V):
    # Rule based tagger
    # specify patterns for tagging
    patterns = [
        (r'.*ing$', 'VERB'),              # gerund
        (r'.*ed$', 'VERB'),               # past tense
        (r'.*es$', 'VERB'),               # 3rd singular present
        (r'.*er$', 'ADJ'),                # adjectives
        (r'.*est$', 'ADJ'),               # adjectives
        (r'.*able$', 'ADJ'),              # adjectives
        (r'.*ly$', 'ADV'),                # adverbs
        (r'.*\'s$', 'NOUN'),              # possessive nouns
        (r'.*s$', 'NOUN'),                # plural nouns
        (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
        (r'\*T?\*?-[0-9]+$', 'X'),        # X
        (r'(The|the|A|a|An|an)$', 'DET'), # articles
        (r'.*', 'NOUN')                   # nouns
    ]
    regexp_tagger = nltk.RegexpTagger(patterns)
    
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        state_max = 'NOUN'
        if (word in vocabulary):
            #initialise list of probability column for a given observation
            p = [] 
            for tag in T:
                if key == 0:
                    transition_p = tags_df.loc['.', tag]
                else:
                    transition_p = tags_df.loc[state[-1], tag]

                # compute emission and state probabilities
                emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
                state_probability = emission_p * transition_p    
                p.append(state_probability)

            pmax = max(p)
            # getting state for which probability is maximum
            state_max = T[p.index(pmax)] 
        else: 
            # Rule based tagging
            pos_tag = regexp_tagger.tag([word])
            state_max = pos_tag[0][1]
            
        state.append(state_max)
    return list(zip(words, state))

In [18]:
# Viterbi Heuristic backed off with probability technique
def Viterbi_backedoff_with_probability_technique(words, train_bag = train_tagged_words, vocabulary=V):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            if (word in vocabulary):
                emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            else:
                emission_p = 0.001
            
            state_probability = emission_p * transition_p
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

#### Evaluating tagging accuracy of modified viterbi using rule based technique

In [19]:
# tagging the validation sentences
start = time.time()
tagged_seq = Viterbi_backedoff_with_rule_based(val_tagged_words)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)

# accuracy
check = [i for i, j in zip(tagged_seq, val_run_base) if i == j]  

accuracy = len(check)/len(tagged_seq)
print("Accuracy of the tagger: ", accuracy)

Time taken in seconds:  862.9099161624908
Accuracy of the tagger:  0.9494393907340808


In [20]:
# Incorrectly tagged words using modified Viterbi based POS tagger using rule based technique
incorrect_tagged_cases = [[val_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, val_run_base)) if j[0]!=j[1]]
incorrect_tagged_cases

[[('to', 'PRT'), (('book', 'NOUN'), ('book', 'VERB'))],
 [('leaving', 'VERB'), (('stocks', 'NOUN'), ('stocks', 'ADV'))],
 [('stocks', 'ADV'), (('up', 'PRT'), ('up', 'ADP'))],
 [('carried', 'VERB'), (('over', 'ADP'), ('over', 'PRT'))],
 [('Chilean', 'ADJ'), (('mine', 'NOUN'), ('mine', 'ADJ'))],
 [('the', 'DET'), (('Palestinian', 'ADJ'), ('Palestinian', 'NOUN'))],
 [('committee', 'NOUN'), (('first', 'ADJ'), ('first', 'ADV'))],
 [('*', 'X'), (('Sit', 'NOUN'), ('Sit', 'VERB'))],
 [('Sit', 'VERB'), (('down', 'ADP'), ('down', 'ADV'))],
 [('The', 'DET'), (('British', 'ADJ'), ('British', 'NOUN'))],
 [('to', 'PRT'), (('halt', 'NOUN'), ('halt', 'VERB'))],
 [('Dow', 'NOUN'), (('slides', 'NOUN'), ('slides', 'VERB'))],
 [('where', 'ADV'), (('most', 'ADJ'), ('most', 'ADV'))],
 [('as', 'ADP'), (('athletic', 'NOUN'), ('athletic', 'ADJ'))],
 [('*-2', 'X'), (('to', 'PRT'), ('to', 'ADJ'))],
 [('1990', 'NUM'), (('better', 'ADJ'), ('better', 'ADV'))],
 [("'s", 'PRT'), (('attempt', 'VERB'), ('attempt', 'NOU

#### Evaluating tagging accuracy of modified viterbi using probability based technique

In [21]:
# tagging the validation sentences
start = time.time()
tagged_seq = Viterbi_backedoff_with_probability_technique(val_tagged_words)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)

# accuracy
check = [i for i, j in zip(tagged_seq, val_run_base) if i == j] 

accuracy = len(check)/len(tagged_seq)
print("Accuracy of the tagger: ", accuracy)

Time taken in seconds:  579.2316379547119
Accuracy of the tagger:  0.9331499894224666


In [22]:
# Incorrectly tagged words using modified Viterbi based POS tagger using probability based technique
incorrect_tagged_cases = [[val_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, val_run_base)) if j[0]!=j[1]]
incorrect_tagged_cases

[[('to', 'PRT'), (('book', 'NOUN'), ('book', 'VERB'))],
 [('leaving', 'VERB'), (('stocks', 'NOUN'), ('stocks', 'ADV'))],
 [('stocks', 'ADV'), (('up', 'PRT'), ('up', 'ADP'))],
 [('carried', 'VERB'), (('over', 'ADP'), ('over', 'PRT'))],
 [('Chilean', 'ADJ'), (('mine', 'NOUN'), ('mine', 'ADJ'))],
 [('the', 'DET'), (('Palestinian', 'ADJ'), ('Palestinian', 'NOUN'))],
 [('committee', 'NOUN'), (('first', 'ADJ'), ('first', 'ADV'))],
 [('has', 'VERB'), (('clamped', 'X'), ('clamped', 'VERB'))],
 [('their', 'PRON'), (('ankle', 'VERB'), ('ankle', 'NOUN'))],
 [('the', 'DET'), (('third-largest', 'NOUN'), ('third-largest', 'ADJ'))],
 [('the', 'DET'), (('fifth-largest', 'NOUN'), ('fifth-largest', 'ADJ'))],
 [('#', '.'), (('89.7', 'NOUN'), ('89.7', 'NUM'))],
 [('$', '.'), (('141.9', 'NOUN'), ('141.9', 'NUM'))],
 [('#', '.'), (('94.8', 'NOUN'), ('94.8', 'NUM'))],
 [('$', '.'), (('149.9', 'NOUN'), ('149.9', 'NUM'))],
 [('The', 'DET'), (('British', 'ADJ'), ('British', 'NOUN'))],
 [('to', 'PRT'), (('halt',

### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

#### Accuracy comparison
1. The accuracy of vanilla Viterbi based POS tagger on validation set is 90.3%
2. The accuracy of modified viterbi using rule based technique on validation set is 95%
3. The accuracy of modified viterbi using probability technique on validation set is 93.3%

### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

#### Observations
1. The unknown words 'outlawing', 'safeguarding' and 'Indexing' are correctly tagged as verbs by modified veterbi algorithms
2. The unknown words like 'broadened', 'shopped' and 'delayed' are correctly tagged as verbs by modified veterbi algorithms
3. The unknown words like 'argues' and 'illustrates' are correctly tagged as verbs by modified veterbi algorithms
4. The unknown words like 'third-largest' and 'finest' are correctly tagged as adjectives by modified veterbi with rule based algorithms
5. The unknown words like 'broadly' and 'faultlessly' are correctly tagged as adverbs
6. The decimal numbers like '89.7' and '141.9' are correctly tagged as 'NUM'

#### Testing on test set

In [23]:
#### Evaluation on test set

sentence_test = """Android is a mobile operating system developed by Google.
Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.
Google and Twitter made a deal in 2015 that gave Google access to Twitter's firehose.
Twitter is an online news and social networking service on which users post and interact with messages known as tweets.
Before entering politics, Donald Trump was a domineering businessman and a television personality.
The 2018 FIFA World Cup is the 21st FIFA World Cup, an international football tournament contested once every four years.
This is the first World Cup to be held in Eastern Europe and the 11th time that it has been held in Europe.
Show me the cheapest round trips from Dallas to Atlanta
I would like to see flights from Denver to Philadelphia.
Show me the price of the flights leaving Atlanta at about 3 in the afternoon and arriving in San Francisco.
NASA invited social media users to experience the launch of ICESAT-2 Satellite."""

words = word_tokenize(sentence_test)

In [24]:
# Evaluation on test set using vanilla viterbi
tagged_seq = Viterbi(words)
print(tagged_seq)

[('Android', 'NUM'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NUM'), ('.', '.'), ('Android', 'NUM'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NUM'), ('worldwide', 'NUM'), ('on', 'ADP'), ('smartphones', 'NUM'), ('since', 'ADP'), ('2011', 'NUM'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NUM'), ('.', '.'), ('Google', 'NUM'), ('and', 'CONJ'), ('Twitter', 'NUM'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'NUM'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'NUM'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'NUM'), ("'s", 'PRT'), ('firehose', 'NUM'), ('.', '.'), ('Twitter', 'NUM'), ('is', 'VERB'), ('an', 'DET'), ('online', 'NUM'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET'), ('users', 'NOU

In [25]:
# Evaluation on test set using modified viterbi using rule based technique
tagged_seq = Viterbi_backedoff_with_rule_based(words)
print(tagged_seq)

[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.'), ('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'VERB'), ('since', 'ADP'), ('2011', 'NUM'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NUM'), ('.', '.'), ('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'ADJ'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'NUM'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'NOUN'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'ADJ'), ("'s", 'PRT'), ('firehose', 'NOUN'), ('.', '.'), ('Twitter', 'ADJ'), ('is', 'VERB'), ('an', 'DET'), ('online', 'NOUN'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET'), ('us

In [26]:
# Evaluation on test set using modified viterbi using probability technique
tagged_seq = Viterbi_backedoff_with_probability_technique(words)
print(tagged_seq)

[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'DET'), ('.', '.'), ('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'DET'), ('since', 'ADP'), ('2011', 'DET'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'DET'), ('.', '.'), ('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'DET'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'X'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'VERB'), ("'s", 'PRT'), ('firehose', 'VERB'), ('.', '.'), ('Twitter', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('online', 'NOUN'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET'), ('user